<a href="https://githubtocolab.com/Eunseob/purdue_me597/blob/main/lab/lab10/L10_Colab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 10.3 Machine Learning 3 - Real-time Sound Recognition for Classification

## Entire remote monitoring system


## 3.1 MTConnect for pump monitoring 

In the prvious section, we performed utilizing the CNN model by TensorFlow Lite to predict the running state of the vacuum pump. Let's make an entire monitoring system as [Figure 1](https://colab.research.google.com/github/Eunseob/purdue_me597/blob/main/lab/lab10/L10_Colab1.ipynb#scrollTo=x_ZERkf6r_E5). Depending on the project or system requirements, you skip the MTConnect part. However, considering the potential use of MTConnect to collect machine data in future, design and create an MTConnect agent to collect and merge the running state prediction (Event) and sound level (Sample) of the vacuum pump.

The idea is that we know both the execution state (OFF/ON, Event) and the vacuum state (Vacuuming/Air leaking, Event) from the CNN model. Also, we calculate the sound level (Sample). The schematic and data items are shown in Figure 7.

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab10_fig7.png?raw=true" width="80%">

*Figure 7 Schematic and data items of MTConnect*

The examples of 'agent.cfg' and 'device.xml' are below. You may have different an XML data structure or data item definitions. Also, you can check the example of MTConnect agent running on the server computer: http://mepotr16.ecn.purdue.edu:5000/. Recall the previous labs to run MTConnect service on Raspberry Pi and reuse the agent execution file and other directory/files (schema, style, and so on).

<br>

---
**<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_conf.png?raw=tru" width="20"> agent.cfg**

```
Devices = device.xml
AllowPut = true
Port = 5000
ReconnectInterval = 1000
BufferSize = 17
SchemaVersion = 1.4
MonitorConfigFiles = true
Pretty = true
# MinimumConfigReloadAge = 30

Adapters {
   # Log file has all machines with device name prefixed
   Device {
      Device = vacuum_pump
      Host = 127.0.0.1
      Port = 7878
   }
}

Files {
    schemas {
        Path = ./schemas
        Location = /schemas/
    }
    styles {
        Path = ./styles
        Location = /styles/
    }
    Favicon {
        Path = ./styles/favicon.ico
        Location = /favicon.ico
    }
}

DevicesStyle { Location = /styles/Devices.xsl }
StreamsStyle { Location = /styles/Streams.xsl }

# Logger Configuration
logger_config
{
    logging_level = debug
}
```
---

<br>

---
**<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_xml.png?raw=tru" width="20"> device.xml**
```
<?xml version="1.0" encoding="UTF-8"?>
<MTConnectDevices xmlns:m="urn:mtconnect.org:MTConnectDevices:1.1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="urn:mtconnect.org:MTConnectDevices:1.1" xsi:schemaLocation="urn:mtconnect.org:MTConnectDevices:1.1 http://www.mtconnect.org/schemas/MTConnectDevices_1.1.xsd">
	<Header creationTime="2010-03-04T18:44:40+00:00" sender="localhost" instanceId="1267728234" bufferSize="131072" version="1.1"/>
	<Devices>
		<Device id="vacuum_pump" name="vacuum_pump" uuid="001">
			<DataItems>
				<DataItem category="EVENT" id="001" name="avail" type="AVAILABILITY"/>
				<DataItem category="EVENT" id="e1" name="execution" type="EXECUTION"/>
				<DataItem category="EVENT" id="vs1" name="vacuum_state" type="STATE"/>
				<DataItem category="SAMPLE" id="spl" name="sound_level" type="SOUND_LEVEL" units="DECIBEL"/>
				
			</DataItems>
		</Device>
	</Devices>
</MTConnectDevices>
```

---

The MTConnect adapter sample code ([lab10_sample_adapter.py]()) is also given. The interpretation algorithm to take the CNN model output as MTConnect data item is shown in Figure 8. If the output is 0, execution is "OFF" and vacuum state is "N/A" (not available). If the output is 1, execution is "ON" and vacuum state "Air leaking". Lastly, if the output is 2, execution is "ON" and vacuum state "Vacuuming". However, it is recommended to try programming the adapter yourself. If you use the given sample code, you have to modify some lines. The incomplete part of the sample script is below. You have to change the variables which is wrarpped by aserteric mark (*). By completing the sample code , perform Task 3.1.

Note that your CNN model (h5 format), [mtconnect_adapter.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab5/adapter_sample/mtconnect_adapter.py?raw=true), and [data_item.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab5/adapter_sample/data_item.py?raw=true) must be in the same directory with the adapter.






### Task 3.1

Run MTConnect agent and adapter on Raspberry Pi with given informationa and sample. Capture '/current' response of the MTConnect agent as Figure 9 from **your laptop** and attach it to the report. While performing this, check if your adapter is working correctly. You see printed out messages in the terminal as Figure 10. 



---

Place your screenshot here.

---



### Task 2.1

Run [lab10_sample2.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab10/lab10_sample_code/lab10_sample2.py?raw=true) on your Raspberry Pi or laptop see if the CNN model works well. The terminal or output shell looks like as Figure 5. **Note that if you develop the CNN model with different feature size and shape, you have to modify the sample code.** An algorithm to interpret the CNN model output is incomplete as part of the sample code below. Complete the algorithm so that appropriate label is printed out at the end. To be specific, if Y (prediction inference index) is 0, the prediction label must be "OFF" (given), if Y is 1, the prediction label must be "ON-airleaking", and so on.

* Hint: Use if, elif, else statement.
* Hint: Another way is to use dictionary data structure with indexing.

<br>

---

**<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_Python.png?raw=tru" width="20">Python - Python3 (Last part of [lab10_sample2.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab10/lab10_sample_code/lab10_sample2.py?raw=true))**

```
91         if Y == 0: # if Y is 0, do below
92             prediction_label = "OFF"
93         # complete your algorithm to take appropriate prediction_label string according to the CNN model result
94 
95         print('The pump is now {}.\n'.format(prediction_label))
96         
97         time.sleep(1) # pause for 1 second

```

---

<br>

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab10_fig5.png?raw=true" width="80%">

*Figure 5 Terminal while running lab10_sample2.py*

In [ ]:
## copy and paste your entire algorithm from line number 91 (if Y == 0: # if Y is 0, do below) to the end



<br></br>

Please continue to [Lab 10.3 here](https://colab.research.google.com/github/Eunseob/purdue_me597/blob/main/lab/lab10/L10_Colab3.ipynb).
